In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
%%sh
cat > pi-mpi.c << EOF

#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// Funkcja do obliczania potęgi
double potegowanie(double a , int b )
{
      double result=1.0;
      for (int i = 1; i <= b; i++)
      {
          result *= a;
      }
      return result; 
}

int main(int argc, char **argv)
{
    int rank; // Numer procesu
    int size; // Liczba procesów
    int tag = 1;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    clock_t t;
    double time_taken;

    if(rank == 0)
    {
      double lpi = 0 ,x = 0,total_time =0,t=0;

      for(int i=1; i<size; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        lpi +=x;
        MPI_Recv(&t, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        total_time+=t;
      }
      printf("Wartość liczby pi = %f \n", 4 * lpi);
      printf("Całkowity czas wykonywania = %f \n", total_time);
    }

    if(rank != 0)
    {
      t = clock();
      // Obliczanie wyniku dla danego procesu
      double x = potegowanie( -1, rank - 1) / ( 2 * rank - 1); 
      printf("Wynik procesu %d = %f \t" , rank , x);
      // Wysyłanie wyniku do procesu 0
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      t = clock() - t;
      time_taken = ((double)t)/CLOCKS_PER_SEC;
      // Wysyłanie informacji o czasie wykonania do procesu 0
      MPI_Send(&time_taken, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      printf("Czas wykonywania sie procesu %d = %f\n",rank,time_taken);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 60 --allow-run-as-root a.out

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// Funkcja, którą calkujemy
double funkcja(double x)
{
  return (x * x + 2 * x + 3*x);
}

int main(int argc, char **argv)
{
  // Inicjalizacja zmiennych potrzebnych do obsługi MPI
    int rank; // Numer procesu
    int size; // Liczba procesów
    int tag=1; // Etykieta wiadomości
    
    MPI_Status status; // Zmienna przechowująca status odebrania wiadomości
    MPI_Init(&argc, &argv); // Inicjalizacja MPI
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Pobranie numeru procesu
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Pobranie liczby procesów
    clock_t t; // Zmienna do mierzenia czasu
    double time_taken; // Zmienna do przechowywania czasu wykonania procesu
    // Zmienne do przechowywania liczby procesów i numeru procesu w formacie double
    double numproc = size, nmbproc = rank;
    // Granice calkowania
    double lower = 1,upper = 4;
    // Krok calkowania
    double h = (upper - lower) / numproc;
    // Proces o numerze 0 jest głównym procesem i odbiera wyniki od pozostałych procesów
    if(rank == 0)
    {
      double calka = 0 ,x=0;
      // Odbieranie wyników od pozostałych procesów
      for(int i=1; i<size; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        calka +=x;
      }
      // Wypisanie wyniku
      printf("Wartosc dolnej granicy calki = %f \n",funkcja(lower)/2 * h);
      printf("Wartosc gornej granicy calki = %f \n",funkcja(upper)/2 * h);

      printf("Wartosc calki = %1f \n", h * (funkcja(lower)/2 + calka 
        + funkcja(upper)/2));
    }

    if(rank != 0)
    {
        t = clock();
        double xi,wynik;
        nmbproc = rank;
        xi = lower + (nmbproc/numproc) 
            * (upper - lower);
        wynik = funkcja(xi);
        printf("Wynik  %d = %f \t", rank,wynik*h);
        MPI_Send(&wynik, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
        t = clock() - t;
        time_taken = ((double)t)/CLOCKS_PER_SEC;
        printf("Czas wykonywania:  %d = %f\n",rank,time_taken);
    }

    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 100 --allow-run-as-root a.out